### Define own method that implements Multidimensional Scaling (MDS)

In [4]:
# Imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances

import numpy as np
from numpy import linalg as LA

import matplotlib.pyplot as plt
from matplotlib import offsetbox

In [3]:
def get_mds(X, dim, typ='C'):
    """ Dimensionality reduction with MDS

    Keyword arguments:
    X -- Data matrix with M rows (observations) and N columns (features)
    dim -- Reduced number of dimensions
    typ -- Type of MDS. 'C' or 'Classical' for CMDS.
    """

    # M observations, N features    
    M = X.shape[0]
    N = X.shape[1]

    # Step 0: Calculate similarities matrix
    # Classical MDS means Euclidean distances
    if typ in ['C', 'Classical']:

        D = pairwise_distances(X, metric='euclidean')
        
    # Only 'C' or 'Classical' are valid inputs
    else:

        print("Only C and Classical are supported inputs for parameter typ") 
        return

    # Step 1: Calculate centering Gram matrix
    # Calculate n-centralizing matrix
    H = np.identity(M) - ((1/M) * np.ones((M,M)))

    # Define centering Gram matrix
    GC = -.5 * (H @ (D**2) @ H)

    # Step 2: Spectral decomposition
    U, S, _ = LA.svd(GC, full_matrices=True)

    # Step 3: Define dimensionality reduction of X
    Y = np.diag(np.sqrt(S[0:dim])) @ U[:,0:dim].T

    return Y

In [2]:
def plot_embedding(X, title):
    """ Plot ISOMAP embedding.

    Keyword arguments:
    X -- Data matrix with M rows (observations) and N columns (features)
    title -- Title of plot
    """
    # Plot
    _, ax = plt.subplots()
    X = MinMaxScaler().fit_transform(X)

    for digit in digits.target_names:
        ax.scatter(
            *X[y == digit].T,
            marker=f"${digit}$",
            s=60,
            color=plt.cm.Dark2(digit),
            alpha=0.425,
            zorder=2,
        )
    shown_images = np.array([[1.0, 1.0]])  # just something big
    for i in range(X.shape[0]):
        # plot every digit on the embedding
        # show an annotation box for a group of digits
        dist = np.sum((X[i] - shown_images) ** 2, 1)
        if np.min(dist) < 4e-3:
            # don't show points that are too close
            continue
        shown_images = np.concatenate([shown_images, [X[i]]], axis=0)
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r), X[i]
        )
        imagebox.set(zorder=1)
        ax.add_artist(imagebox)

    ax.set_title(title)
    ax.axis("off")